In [1]:
import functools

In [2]:
import iteration_utilities

In [3]:
import random

In [4]:
import numpy as np

In [5]:
import math

In [6]:
import tensorflow as tf

In [7]:
from tensorflow.python import debug as tf_debug

In [8]:
from enum import IntEnum

In [9]:
print("TensorFlow version: {}".format(tf.VERSION))

TensorFlow version: 1.8.0


In [10]:
# (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
training_data, testing_data = tf.keras.datasets.fashion_mnist.load_data()

In [11]:
preferred_number_samples_per_layer = 16

In [12]:
number_of_classes = np.max(testing_data[1]).__int__() + 1
number_of_input_channels = 1

In [13]:
number_of_intermediate_layers = 500

In [14]:
number_of_computing_layers = number_of_intermediate_layers + number_of_classes

In [15]:
number_of_layers = number_of_input_channels + number_of_computing_layers

In [16]:
paddings = (0, 0), (2, 2), (2, 2)

In [17]:
random_seed = number_of_intermediate_layers

In [18]:
def variable_arrays(prefix):
    random.seed(random_seed)
    indicies = tuple(
        tuple(
            source_index for source_index
            in range(target_index)
            if random.randrange(target_index) < preferred_number_samples_per_layer
        ) for target_index
        in range(number_of_layers)
    )
    flatten_source_indicies = tuple(iteration_utilities.flatten(indicies))
    lengths = tuple(map(len, indicies))
    return tf.TensorArray(tf.int32, size=number_of_layers, clear_after_read=False, infer_shape=False).split(
        tf.get_variable(
            name=f"{prefix}_source_index",
            initializer=flatten_source_indicies,
            dtype=tf.int32,
        ),
        lengths,
    ), tf.TensorArray(tf.float32, size=number_of_layers, clear_after_read=False, infer_shape=False).split(
        tf.get_variable(
            name=f"{prefix}_weight",
            initializer=tf.variance_scaling_initializer(),
            dtype=tf.float32,
            shape=(len(flatten_source_indicies), )
        ),
        lengths,
    )

In [19]:
def grid_sample(images, offset_x, offset_y):
    image_shape = tf.shape(images)
    current_batch_size = image_shape[0]
    width = image_shape[1]
    height = image_shape[2]
    
    def check_width(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, width - 1), 0)
    
    def check_height(tensor):
        if (tf.test.is_gpu_available()):
            return tensor
        else:
            return tf.maximum(tf.minimum(tensor, height - 1), 0)
    
    offset_left = tf.floor(offset_x)
    offset_top = tf.floor(offset_y)
    tf.assert_equal(tf.shape(offset_left), image_shape)
    tf.assert_equal(tf.shape(offset_top), image_shape)

    
    eps = 1e-7

    factor_right = offset_x - offset_left + eps
    factor_left = 1.0 + 2.0 * eps - factor_right
    factor_bottom = offset_y - offset_top + eps
    factor_top = 1.0 + 2.0 * eps - factor_bottom
    
    image_index, x_index, y_index = tf.meshgrid(
        tf.range(current_batch_size, dtype=tf.int32),
        tf.range(width, dtype=tf.int32),
        tf.range(height, dtype=tf.int32),
        indexing='ij',
    )
    
    tf.assert_equal(tf.shape(image_index), image_shape)
    tf.assert_equal(tf.shape(x_index), image_shape)
    tf.assert_equal(tf.shape(y_index), image_shape)

    
    left_index = check_width(x_index + tf.cast(offset_left, tf.int32))
    top_index = check_height(y_index + tf.cast(offset_top, tf.int32))

    right_index = left_index + 1
    bottom_index = top_index + 1
    
    tf.assert_equal(tf.shape(left_index), image_shape)
    tf.assert_equal(tf.shape(top_index), image_shape)
    tf.assert_equal(tf.shape(right_index), image_shape)
    tf.assert_equal(tf.shape(bottom_index), image_shape)    
    
    tf.assert_rank(left_index, 3)
    tf.assert_rank(right_index, 3)
    tf.assert_rank(bottom_index, 3)
    tf.assert_rank(top_index, 3)
    
    images_top_left = tf.gather_nd(images, tf.stack((image_index, left_index, top_index), axis=3))
    images_top_right = tf.gather_nd(images, tf.stack((image_index, right_index, top_index), axis=3))
    images_bottom_left = tf.gather_nd(images, tf.stack((image_index, left_index, bottom_index), axis=3))
    images_bottom_right = tf.gather_nd(images, tf.stack((image_index, right_index, bottom_index), axis=3))

    tf.assert_rank(images_top_left, 3)
    tf.assert_rank(images_top_right, 3)
    tf.assert_rank(images_bottom_left, 3)
    tf.assert_rank(images_bottom_right, 3)

    lerp_top = factor_left * images_top_left + factor_right * images_top_right
    lerp_bottom = factor_left * images_bottom_left + factor_right * images_bottom_right
    output = factor_top * lerp_top + factor_bottom * lerp_bottom
    tf.assert_rank(output, 3)
    return output

In [20]:
def model_fn(features, labels, mode, params, config):
    padded_input = tf.pad(tf.cast(features, tf.float32) / 255.0 - 0.5, paddings)
    tf.assert_type(padded_input, tf.float32)
    offset_x_source_index_array, offset_x_weight_array = variable_arrays("offset_x")
    offset_x_bias_array = tf.get_variable(
        name="offset_x_bias",
        initializer=tf.random_uniform_initializer(minval=-3.0, maxval=3.0),
        dtype=tf.float32,
        shape=(number_of_layers, ),
    )
    offset_y_source_index_array, offset_y_weight_array = variable_arrays("offset_y")
    offset_y_bias_array = tf.get_variable(
        name="offset_y_bias",
        initializer=tf.random_uniform_initializer(minval=-3.0, maxval=3.0),
        dtype=tf.float32,
        shape=(number_of_layers, ),
    )
    
    score_source_index_array, score_weight_array = variable_arrays("score")
    score_bias_array = tf.get_variable(
        name="score_bias",
        initializer=tf.zeros_initializer(),
        dtype=tf.float32,
        shape=(number_of_layers, ),
    )
    
    def score_map(previous_layers, i):
        def indexed_sum(source_index_array, weight_array, bias_array):
            """
            TODO: Add a factor multiplied for entire layer
            """
            
            product = bias_array[i] + tf.reduce_sum(
                previous_layers.gather(source_index_array.read(i)) * functools.reduce(tf.expand_dims, range(1, 4), weight_array.read(i)),
                axis=0
            )
            return product
        return grid_sample(
            indexed_sum(score_source_index_array, score_weight_array, score_bias_array),
            indexed_sum(offset_x_source_index_array, offset_x_weight_array, offset_x_bias_array),
            indexed_sum(offset_y_source_index_array, offset_y_weight_array, offset_y_bias_array),
        )

    layers, i = tf.while_loop(
        lambda layers, i: i < number_of_layers,
        lambda layers, i: (layers.write(i, score_map(layers, i)), i + 1),
        tf.while_loop(
            lambda layers, i: i < number_of_input_channels + number_of_intermediate_layers,
            lambda layers, i: (layers.write(i, tf.nn.relu(score_map(layers, i))), i + 1),
            (
                tf.TensorArray(tf.float32, size=number_of_layers, clear_after_read=False, infer_shape=True).write(
                    0,
                    padded_input,
                ),
                tf.constant(number_of_input_channels),
            )
        )
    )
    tf.assert_equal(i, tf.cast(number_of_layers, tf.int32))

    scores_per_pixel = layers.gather(tf.range(number_of_layers - number_of_classes, number_of_layers, dtype=tf.int32))
    score_shape = tf.shape(scores_per_pixel)
    scores = tf.transpose(scores_per_pixel[:, :, score_shape[2] // 2, score_shape[3] // 2])
    probabilities = tf.nn.softmax(logits=scores)
    predicted_classes = tf.argmax(scores, 1)
    predictions = {
        'probabilities' : probabilities,
        'scores': scores,
        'class': predicted_classes,
    }
    eval_metric_ops = {
        'accuracy': tf.metrics.accuracy(labels=labels, predictions=predicted_classes)
    }
    loss = tf.losses.softmax_cross_entropy(logits=scores, onehot_labels=tf.one_hot(labels, number_of_classes))
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops
    )

In [21]:
run_config = tf.estimator.RunConfig(
    model_dir=f"models/reluoffnet{number_of_intermediate_layers}",
    session_config=tf.ConfigProto(
        graph_options=tf.GraphOptions(
            optimizer_options=tf.OptimizerOptions(
                global_jit_level=tf.OptimizerOptions.ON_2,
                do_function_inlining=True,
                do_constant_folding=True,
                do_common_subexpression_elimination=True,
            )
        )
    )
)

In [22]:
estimator = tf.estimator.Estimator(model_fn, config=run_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/reluoffnet500', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': graph_options {
  optimizer_options {
    do_common_subexpression_elimination: true
    do_constant_folding: true
    do_function_inlining: true
    global_jit_level: ON_2
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f66ae107320>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [23]:
batch_size = 64

In [24]:
def training_dataset():
    return tf.data.Dataset.from_tensor_slices(training_data).shuffle(1000).repeat().batch(batch_size)

In [25]:
def testing_dataset():
    return tf.data.Dataset.from_tensor_slices(testing_data).batch(batch_size)

In [26]:
# hook = tf_debug.TensorBoardDebugHook("localhost:6064")
# estimator.train(training_dataset,hooks=[hook])

In [ ]:
tf.estimator.train_and_evaluate(
    estimator,
    train_spec=tf.estimator.TrainSpec(training_dataset),
    eval_spec=tf.estimator.EvalSpec(testing_dataset)
)

INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 600 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/reluoffnet500/model.ckpt.
INFO:tensorflow:loss = 2.3025823, step = 0
